In [1]:
import numpy as np
import pandas as pd
import requests

In [14]:
def url_from_page(page: int) -> str:
    offset = (page - 1) * 50
    return f"https://www.hltv.org/stats/matches?startDate=all&offset={offset}"

data_frames = []

for page in range(1, 4):
    url = url_from_page(page)

    response = requests.get(url)

    df = pd.read_html(response.text)
    data_frames.append(pd.concat(df))

teams = pd.concat(data_frames)
teams

,Date,Team1,Team2,Map,Event
0,19/2/23,EG White (16),Villainous (12),mrg Mirage,ESEA Winter 2023 Cash Cup 3 North America
1,19/2/23,paiN (4),caffeine marauders (16),inf Inferno,ESEA Winter 2023 Cash Cup 3 North America
2,19/2/23,Fuscão 1500 (16),Corinthians (11),ovp Overpass,BLAST.tv Paris Major 2023 South America RMR Cl...
3,19/2/23,Corinthians (14),Fuscão 1500 (16),vtg Vertigo,BLAST.tv Paris Major 2023 South America RMR Cl...
4,19/2/23,ODDIK (11),9z (16),ovp Overpass,BLAST.tv Paris Major 2023 South America RMR Cl...
...,...,...,...,...,...
45,18/2/23,Sangal (16),MASONIC (10),ovp Overpass,European Development Championship 7
46,18/2/23,Viperio (16),PGE Turow (13),anc Ancient,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...
47,18/2/23,LDLC (6),B8 (16),nuke Nuke,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...
48,18/2/23,Anonymo (12),JANO (16),mrg Mirage,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...


In [45]:
import re

def get_score(value: str) -> int:
    return int(re.search(r'\((\d+)\)', value).group(1))

teams['Team1 Score'] = list(map(get_score, teams['Team1'].tolist()))
teams['Team2 Score'] = list(map(get_score, teams['Team2'].tolist()))

In [52]:
# Remove score from team name

def get_team(value: str) -> str:
    return re.search(r'.+?(?=\s\()', value).group()

teams['Team1'] = list(map(get_team, teams['Team1'].tolist()))
teams['Team2'] = list(map(get_team, teams['Team2'].tolist()))
teams

,Date,Team1,Team2,Map,Event,Team1 Score,Team2 Score,Team1 Won,Team2 Won
0,19/2/23,EG White,Villainous,mrg Mirage,ESEA Winter 2023 Cash Cup 3 North America,16,12,True,False
1,19/2/23,paiN,caffeine marauders,inf Inferno,ESEA Winter 2023 Cash Cup 3 North America,4,16,False,True
2,19/2/23,Fuscão 1500,Corinthians,ovp Overpass,BLAST.tv Paris Major 2023 South America RMR Cl...,16,11,True,False
3,19/2/23,Corinthians,Fuscão 1500,vtg Vertigo,BLAST.tv Paris Major 2023 South America RMR Cl...,14,16,False,True
4,19/2/23,ODDIK,9z,ovp Overpass,BLAST.tv Paris Major 2023 South America RMR Cl...,11,16,False,True
...,...,...,...,...,...,...,...,...,...
45,18/2/23,Sangal,MASONIC,ovp Overpass,European Development Championship 7,16,10,True,False
46,18/2/23,Viperio,PGE Turow,anc Ancient,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...,16,13,True,False
47,18/2/23,LDLC,B8,nuke Nuke,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...,6,16,False,True
48,18/2/23,Anonymo,JANO,mrg Mirage,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...,12,16,False,True


In [53]:
teams['Team1 Won'] = teams['Team1 Score'] > teams['Team2 Score']
teams['Team2 Won'] = teams['Team1 Score'] < teams['Team2 Score']
teams

,Date,Team1,Team2,Map,Event,Team1 Score,Team2 Score,Team1 Won,Team2 Won
0,19/2/23,EG White,Villainous,mrg Mirage,ESEA Winter 2023 Cash Cup 3 North America,16,12,True,False
1,19/2/23,paiN,caffeine marauders,inf Inferno,ESEA Winter 2023 Cash Cup 3 North America,4,16,False,True
2,19/2/23,Fuscão 1500,Corinthians,ovp Overpass,BLAST.tv Paris Major 2023 South America RMR Cl...,16,11,True,False
3,19/2/23,Corinthians,Fuscão 1500,vtg Vertigo,BLAST.tv Paris Major 2023 South America RMR Cl...,14,16,False,True
4,19/2/23,ODDIK,9z,ovp Overpass,BLAST.tv Paris Major 2023 South America RMR Cl...,11,16,False,True
...,...,...,...,...,...,...,...,...,...
45,18/2/23,Sangal,MASONIC,ovp Overpass,European Development Championship 7,16,10,True,False
46,18/2/23,Viperio,PGE Turow,anc Ancient,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...,16,13,True,False
47,18/2/23,LDLC,B8,nuke Nuke,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...,6,16,False,True
48,18/2/23,Anonymo,JANO,mrg Mirage,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...,12,16,False,True


In [56]:
winners = []

for index, row in teams.iterrows():
    if row['Team1 Won']:
        winners.append(row['Team1'])
    else:
        winners.append(row['Team2'])

teams['Winner'] = winners
teams

,Date,Team1,Team2,Map,Event,Team1 Score,Team2 Score,Team1 Won,Team2 Won,Winner
0,19/2/23,EG White,Villainous,mrg Mirage,ESEA Winter 2023 Cash Cup 3 North America,16,12,True,False,EG White
1,19/2/23,paiN,caffeine marauders,inf Inferno,ESEA Winter 2023 Cash Cup 3 North America,4,16,False,True,caffeine marauders
2,19/2/23,Fuscão 1500,Corinthians,ovp Overpass,BLAST.tv Paris Major 2023 South America RMR Cl...,16,11,True,False,Fuscão 1500
3,19/2/23,Corinthians,Fuscão 1500,vtg Vertigo,BLAST.tv Paris Major 2023 South America RMR Cl...,14,16,False,True,Fuscão 1500
4,19/2/23,ODDIK,9z,ovp Overpass,BLAST.tv Paris Major 2023 South America RMR Cl...,11,16,False,True,9z
...,...,...,...,...,...,...,...,...,...,...
45,18/2/23,Sangal,MASONIC,ovp Overpass,European Development Championship 7,16,10,True,False,Sangal
46,18/2/23,Viperio,PGE Turow,anc Ancient,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...,16,13,True,False,Viperio
47,18/2/23,LDLC,B8,nuke Nuke,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...,6,16,False,True,B8
48,18/2/23,Anonymo,JANO,mrg Mirage,BLAST.tv Paris Major 2023 Europe RMR Closed Qu...,12,16,False,True,JANO


In [57]:
full_teams = teams.copy(deep=True)
teams = teams[['Team1', 'Team2', 'Team1 Score', 'Team2 Score', 'Winner', 'Map']]
teams

,Team1,Team2,Team1 Score,Team2 Score,Winner,Map
0,EG White,Villainous,16,12,EG White,mrg Mirage
1,paiN,caffeine marauders,4,16,caffeine marauders,inf Inferno
2,Fuscão 1500,Corinthians,16,11,Fuscão 1500,ovp Overpass
3,Corinthians,Fuscão 1500,14,16,Fuscão 1500,vtg Vertigo
4,ODDIK,9z,11,16,9z,ovp Overpass
...,...,...,...,...,...,...
45,Sangal,MASONIC,16,10,Sangal,ovp Overpass
46,Viperio,PGE Turow,16,13,Viperio,anc Ancient
47,LDLC,B8,6,16,B8,nuke Nuke
48,Anonymo,JANO,12,16,JANO,mrg Mirage


In [67]:
team_1_df = teams[['Team1', 'Team1 Score', 'Winner', 'Map', 'Team2']].rename(columns={
    'Team1': 'Team',
    'Team1 Score': 'Score',
    'Team2': 'Opponent',
})
team_1_df['Is Winner'] = team_1_df['Team'] == team_1_df['Winner']

team_2_df = teams[['Team2', 'Team2 Score', 'Winner', 'Map', 'Team1']].rename(columns={
    'Team2': 'Team',
    'Team2 Score': 'Score',
    'Team1': 'Opponent',
})
team_2_df['Is Winner'] = team_2_df['Team'] == team_2_df['Winner']

teams = pd.concat([team_1_df, team_2_df])
teams

,Team,Score,Winner,Map,Opponent,Is Winner
0,EG White,16,EG White,mrg Mirage,Villainous,True
1,paiN,4,caffeine marauders,inf Inferno,caffeine marauders,False
2,Fuscão 1500,16,Fuscão 1500,ovp Overpass,Corinthians,True
3,Corinthians,14,Fuscão 1500,vtg Vertigo,Fuscão 1500,False
4,ODDIK,11,9z,ovp Overpass,9z,False
...,...,...,...,...,...,...
45,MASONIC,10,Sangal,ovp Overpass,Sangal,False
46,PGE Turow,13,Viperio,anc Ancient,Viperio,False
47,B8,16,B8,nuke Nuke,LDLC,True
48,JANO,16,JANO,mrg Mirage,Anonymo,True


In [94]:
sum_teams = teams.groupby('Team').sum('Score')
sum_teams

,Score,Is Winner
Team,,
00NATION,85,4
1WIN,32,2
4ORMULA,13,0
500,111,4
9z,63,3
...,...,...
iNation,80,5
los kogutos,32,2
paiN,36,2


In [95]:
sum_teams['Matches'] = teams.groupby('Team').size()
sum_teams

,Score,Is Winner,Matches
Team,,,
00NATION,85,4,5
1WIN,32,2,2
4ORMULA,13,0,2
500,111,4,8
9z,63,3,5
...,...,...,...
iNation,80,5,5
los kogutos,32,2,2
paiN,36,2,3


In [97]:
sum_teams['Winrate'] = sum_teams['Is Winner'] * 100 / sum_teams['Matches']
sum_teams

,Score,Is Winner,Matches,Winrate
Team,,,,
00NATION,85,4,5,80.000000
1WIN,32,2,2,100.000000
4ORMULA,13,0,2,0.000000
500,111,4,8,50.000000
9z,63,3,5,60.000000
...,...,...,...,...
iNation,80,5,5,100.000000
los kogutos,32,2,2,100.000000
paiN,36,2,3,66.666667
